# Linux专项

## 1.概念引入

正式讲课之前，先说些基本概念，难理解的用程序跑跑然后再理解：**如有错误欢迎批评指正**

**<a href="https://baike.baidu.com/item/并发" target="_blank">并发</a>** :一个时间段中有几个程序都处于已启动运行到运行完毕之间，且这几个程序都是在同一个处理机上运行，但任一个时刻点上只有一个程序在处理机上运行。

**<a href="https://baike.baidu.com/item/并行" target="_blank">并行</a>** :当一个CPU执行一个线程时，另一个CPU可以执行另一个线程，两个线程互不抢占CPU资源，可以同时进行，这种方式我们称之为并行(Parallel)。（在同一个时间段内，两个或多个程序执行，有时间上的重叠）

---

通俗的举个例子：

小明、小潘、小张、小康去食堂打饭，4个小伙子Coding了3天，饿爆了，现在需要1分钟内让他们都吃上饭，不然就有可怕的事情发生。

按照正常的流程，1分钟可能只够他们一个人打饭，这不行啊，于是有了几种处理方法：

**并发**：快快快，**一人先吃一口，轮着来，一直喂到你们都饱了**(只有一个食堂打饭的窗口)（单核CPU）

**并行**：
- 开了2~3个窗口，和上面处理一样，只是竞争的强度没那么大了
- 开了4个窗口，不着急，一人一个窗口妥妥的

---

对于操作系统来说，**一个任务就是一个进程（Process）**，比如打开一个浏览器就是启动一个浏览器进程，打开两个浏览器就启动了两个浏览器进程。

有些进程还不止同时干一件事，比如Word，它可以同时进行打字、拼写检查、打印等事情。**在一个进程内部，要同时干多件事，就需要同时运行多个“子任务”，我们把进程内的这些“子任务”称为线程（Thread）**。

由于**每个进程至少要干一件事，所以，一个进程至少有一个线程**。像Word这种复杂的进程可以有多个线程，多个线程可以同时执行，多线程的执行方式和多进程是一样的，也是由操作系统在多个线程之间快速切换，让每个线程都短暂地交替运行，看起来就像同时执行一样。

通俗讲：**线程是最小的执行单元，而进程由至少一个线程组成。`如何调度进程和线程，完全由操作系统决定，程序自己不能决定什么时候执行，执行多长时间`**

---

PS：进程5态下次正式讲程序的时候会说，然后就是==> 程序实战不会像今天这样繁琐的，Code很简单，但是不懂这些基本概念往后会吃很多亏，逆天遇到太多坑了，所以避免大家入坑，简单说说这些概念和一些偏底层的东西～看不懂没事，有个印象即可，以后遇到问题至少知道从哪个方向去解决

## 2.进程相关

### 2.1.fork引入

(linux/unix)操作系统提供了一个`fork()`系统调用。普通的函数调用，调用一次，返回一次，但是`fork()`**一次调用，两次返回**。

因为操作系统自动把父进程复制了一份，分别在父进程和子进程内返回。为了便于区分，操作系统是这样做的：**子进程永远返回0，而父进程返回子进程的ID**

查看下帮助文档：
```py
import os

help(os.fork)
Help on built-in function fork in module posix:

fork()
    Fork a child process.
    
    Return 0 to child process and PID of child to parent process.
```
我们来跑个程序验证一下：（PID返回值如果小于0一般都是出错了）
```py
import os

def main():
    print("准备测试～PID：%d" % os.getpid())
    pid = os.fork()
    if pid == 0:
        print("子进程：PID：%d,PPID：%d" % (os.getpid(), os.getppid()))
    elif pid > 0:
        print("父进程：PID：%d,PPID：%d" % (os.getpid(), os.getppid()))

if __name__ == '__main__':
    main()
```
结果：
```
准备测试～PID：11247
父进程：PID：11247,PPID：11229
子进程：PID：11248,PPID：11247
```
可以查看下这个进程是啥：
![PPID.png](https://images2018.cnblogs.com/blog/1127869/201807/1127869-20180725060305297-1803214924.png)

这个指令如果还不熟悉，Linux基础得好好复习下了：<a href="https://www.cnblogs.com/dunitian/p/4822807.html" target="_blank">https://www.cnblogs.com/dunitian/p/4822807.html</a>，简单分析下吧：a是查看所有（可以联想ls -a），u是显示详细信息，x是把不依赖终端的进程也显示出来（终端可以理解为：人与机器交互的那些）

技巧：指令学习可以递增式学习：`ps`，`ps a` `ps au` `ps aux` `ps ajx`

现在验证一下复制一份是什么意思：（代码原封不动，只是在最下面添加了一行输出）
```py
import os

def main():
    print("准备测试～PID：%d" % os.getpid())
    pid = os.fork() # 子进程被父进程fork出来后，在fork处往下执行
    
    if pid == 0:
        print("子进程：PID：%d,PPID：%d" % (os.getpid(), os.getppid()))
    elif pid > 0:
        print("父进程：PID：%d,PPID：%d" % (os.getpid(), os.getppid()))
    
    print("PID:%d,我是卖报的小行家，大风大雨都不怕" % os.getpid())

if __name__ == '__main__':
    main()
```
输出：（父子进程的执行顺序是系统调度决定的）
```py
准备测试～PID：13081
父进程：PID：13081,PPID：9388
PID:13081,我是卖报的小行家，大风大雨都不怕
子进程：PID：13083,PPID：13081
PID:13083,我是卖报的小行家，大风大雨都不怕
```
的确是Copy了一份，Code都一样（玩过逆向的应该知道，**这份Code其实就放在了`.text`（代码段）里面**）

子进程被父进程fork出来后，在fork处往下执行（Code和父进程一样），这时候他们就为了抢CPU各自为战了

最后验证一下：**各个进程地址空间中数据是完全独立的**（有血缘关系的则是：读时共享，写时复制，比如父子进程等）
```py
import os

def main():
    num = 100
    pid = os.fork()
    # 子进程
    if pid == 0:
        num += 10
    elif pid > 0:
        num += 20

    print("PID:%d，PPID：%d，Num=%d" % (os.getpid(), os.getppid(), num))

if __name__ == '__main__':
    main()
```
输出：（进程间通信下一节课会系统的讲，今天只谈Linux和概念）
```
PID:6369，PPID：6332，Num=120
PID:6376，PPID：6369，Num=110
```

---

扩展：（简单了解下即可）
1. **程序**：二进制文件（占用磁盘）
2. **进程**：启动的程序（所有数据都在内存中，需要占用CPU、内存等资源）
3. 进程是CPU、内存、I/0设备的抽象（**各个进程地址空间中数据是完全独立的**）
4. 0号进程是Linux**内核进程**（这么理解：初代吸血鬼）
5. 1号进程是用户进程，所有进程的创建或多或少和它有关系（`init` or `systemd`）
6. 2号进程和1号进程一样，都是0号进程创建的，所有**线程调度**都和他有关系

先看看Linux启动的图示：（图片来自网络）
![2.Linux_Start](https://images2018.cnblogs.com/blog/1127869/201807/1127869-20180725075809773-478928450.png)

查看一下init进程
![Ubuntu](https://images2018.cnblogs.com/blog/1127869/201807/1127869-20180725081532394-1503236514.png)

CentOS进行了优化管理～`systemd`
![CentOS](https://images2018.cnblogs.com/blog/1127869/201807/1127869-20180725081540780-1977663229.png)

其实程序开机启动方式也可以知道区别了：`systemctl start mongodb.service` and `sudo /etc/init.d/ssh start`

Win系列的0号进程：
![win](https://images2018.cnblogs.com/blog/1127869/201807/1127869-20180725081524339-1952364091.jpg)

---

第5点的说明：（以远程CentOS服务器为例）
**pstree -ps**
```shell
systemd(1)─┬─NetworkManager(646)─┬─{NetworkManager}(682)
           │                     └─{NetworkManager}(684)
           ├─agetty(1470)
           ├─auditd(600)───{auditd}(601)
           ├─crond(637)
           ├─dbus-daemon(629)───{dbus-daemon}(634)
           ├─firewalld(645)───{firewalld}(774)
           ├─lvmetad(483)
           ├─master(1059)─┬─pickup(52930)
           │              └─qmgr(1061)
           ├─polkitd(627)─┬─{polkitd}(636)
           │              ├─{polkitd}(639)
           │              ├─{polkitd}(640)
           │              ├─{polkitd}(642)
           │              └─{polkitd}(643)
           ├─rsyslogd(953)─┬─{rsyslogd}(960)
           │               └─{rsyslogd}(961)
           ├─sshd(950)───sshd(50312)───sshd(50325)───bash(50326)───pstree(54258)
           ├─systemd-journal(462)
           ├─systemd-logind(626)
           ├─systemd-udevd(492)
           └─tuned(951)─┬─{tuned}(1005)
                        ├─{tuned}(1006)
                        ├─{tuned}(1007)
                        └─{tuned}(1048)
```
再看一个例子：
```shell
[dnt@localhost ~]$ pstree dnt -ps
sshd(50325)───bash(50326)───pstree(54471)
[dnt@localhost ~]$ pstree 50325 -ps
systemd(1)───sshd(950)───sshd(50312)───sshd(50325)───bash(50326)───pstree(54489)
```
其实你可以在虚拟机试试干死1号进程，就到了登录页面了【现在大部分系统都不让你这么干了】
**kill -9 1**
```
-bash: kill: (1) - 不允许的操作
```

### 2.2.僵尸进程和孤儿进程

先看看定义：

**<a href="https://baike.baidu.com/item/孤儿进程" target="_blank">孤儿进程</a>** ：一个父进程退出，而它的一个或多个子进程还在运行，那么那些子进程将成为孤儿进程。孤儿进程将被init进程(进程号为1)所收养，并由init进程对它们完成状态收集工作。

**<a href="https://baike.baidu.com/item/僵尸进程" target="_blank">僵尸进程</a>** ：一个进程使用fork创建子进程，如果子进程退出，而父进程并没有调用wait或waitpid获取子进程的状态信息，那么子进程的进程描述符仍然保存在系统中。这种进程称之为僵死进程。

通俗讲就是：

**孤儿进程：你爸在你之前死了，你成了孤儿，然后你被进程1收养，你死后的事宜你干爹帮你解决**

**僵尸进程：你挂了，你爸忙着干其他事情没有帮你安葬，你变成了孤魂野鬼，你的怨念一直长存世间**

举个例子看看：
```py
import os
import time

def main():
    pid = os.fork()
    if pid == 0:
        print("子进程：Pid=%d,PPID=%d" % (os.getpid(), os.getppid()))
        time.sleep(1)  # 睡1s
    elif pid > 0:
        print("父进程：Pid=%d,PPID=%d" % (os.getpid(), os.getppid()))

    print("pid=%d,over" % os.getpid())

if __name__ == '__main__':
    main()
```
输出：
![孤儿](https://images2018.cnblogs.com/blog/1127869/201807/1127869-20180725085407754-881176336.png)

```py
import os
import time

def main():
    pid = os.fork()
    if pid == 0:
        print("子进程：Pid=%d,PPID=%d" % (os.getpid(), os.getppid()))
    elif pid > 0:
        print("父进程：Pid=%d,PPID=%d" % (os.getpid(), os.getppid()))
        while 1:
            print("父亲我忙着呢，没时间管你个小屁孩")
            time.sleep(1)

    print("pid=%d,over" % os.getpid())

if __name__ == '__main__':
    main()
```
输出+测试：
![僵尸](https://images2018.cnblogs.com/blog/1127869/201807/1127869-20180725090628351-1966112474.png)

其实僵尸进程的危害真的很大，这也就是为什么有些人为了追求效率过度调用底层，不考虑自己实际情况最后发现还不如用自托管的效率高

僵尸进程是杀不死的，必须杀死父类才能彻底解决它们，下面说说怎么让父进程为子进程‘收尸’

---

### 2.3.父进程回收子进程（wait and waitpid）

讲解之前先简单分析一下上面的Linux指令（防止有人不太清楚）

**`kill -9 pid`** ==> 以前逆天说过，是无条件杀死进程，其实这种说法不准确，应该是**发信号给某个进程**

-9指的就是信号道里面的`SIGKILL`（信号终止），你写成`kill -SIGKILL pid`也一样

-9只是系统给的一种简化写法（好像记得1~31信号，各个Linux中都差不多，其他的有点不一样）
```shell
dnt@MZY-PC:~/桌面/work/PC/python/Thread/Linux kill -l
 1) SIGHUP	 2) SIGINT	 3) SIGQUIT	 4) SIGILL	 5) SIGTRAP
 6) SIGABRT	 7) SIGBUS	 8) SIGFPE	 9) SIGKILL	10) SIGUSR1
11) SIGSEGV	12) SIGUSR2	13) SIGPIPE	14) SIGALRM	15) SIGTERM
16) SIGSTKFLT	17) SIGCHLD	18) SIGCONT	19) SIGSTOP	20) SIGTSTP
21) SIGTTIN	22) SIGTTOU	23) SIGURG	24) SIGXCPU	25) SIGXFSZ
26) SIGVTALRM	27) SIGPROF	28) SIGWINCH	29) SIGIO	30) SIGPWR
31) SIGSYS	34) SIGRTMIN	35) SIGRTMIN+1	36) SIGRTMIN+2	37) SIGRTMIN+3
38) SIGRTMIN+4	39) SIGRTMIN+5	40) SIGRTMIN+6	41) SIGRTMIN+7	42) SIGRTMIN+8
43) SIGRTMIN+9	44) SIGRTMIN+10	45) SIGRTMIN+11	46) SIGRTMIN+12	47) SIGRTMIN+13
48) SIGRTMIN+14	49) SIGRTMIN+15	50) SIGRTMAX-14	51) SIGRTMAX-13	52) SIGRTMAX-12
53) SIGRTMAX-11	54) SIGRTMAX-10	55) SIGRTMAX-9	56) SIGRTMAX-8	57) SIGRTMAX-7
58) SIGRTMAX-6	59) SIGRTMAX-5	60) SIGRTMAX-4	61) SIGRTMAX-3	62) SIGRTMAX-2
63) SIGRTMAX-1	64) SIGRTMAX	
```
一般搜索进程中的某个程序一般都是用这个：**`ps -aux | grep xxx`** （`|`其实就是管道，用于有血缘关系进程间通信，等会讲）

如果安装了`pstree`就更方便了：**`pstree 13570 -ps`** （Ubuntu自带，CentOS要装下`yum install psmisc`）
```py
systemd(1)───systemd(1160)───gnome-terminal-(21604)───bash(8169)───python3(13570)───python3(13571)
```

---

#### 2.3.1.wait

步入正题：

Python的Wait和C系列的稍有不同，这边重点说说Python：
```py
help(os.wait)

Help on built-in function wait in module posix:

wait()
    Wait for completion of a child process.
    
    Returns a tuple of information about the child process:
        (pid, status)
```
`os.wait()`返回一个元组，第一个是进程id，第二个是状态，正常退出是0，被九号信号干死就返回9

来个案例：
```py
import os
import time

def main():
    pid = os.fork()
    if pid == 0:
        print("子进程：Pid=%d,PPID=%d" % (os.getpid(), os.getppid()))
    elif pid > 0:
        print("父进程：Pid=%d,PPID=%d" % (os.getpid(), os.getppid()))
        wpid, status = os.wait()
        print(wpid)
        print(status)

    print("pid=%d,over" % os.getpid())

if __name__ == '__main__':
    main()
```
输出：
```
父进程：Pid=22322,PPID=10139
子进程：Pid=22323,PPID=22322
pid=22323,over
22323
0
pid=22322,over
```
演示一下被9号信号干死的情况：
```py
import os
import time

def main():
    pid = os.fork()
    if pid == 0:
        print("子进程：Pid=%d,PPID=%d" % (os.getpid(), os.getppid()))
        while 1:
            print("孩子老卵，就是不听话")
            time.sleep(1)
    elif pid > 0:
        print("父进程：Pid=%d,PPID=%d" % (os.getpid(), os.getppid()))
        wpid, status = os.wait()  # 调用一次只能回收一次，想都回收，就来个while循环，-1则退出
        print(wpid)
        print(status)
        if status == 0:
            print("正常退出")
        elif status == 9:
            print("被信号9干死了")

    print("pid=%d,over" % os.getpid())

if __name__ == '__main__':
    main()
```
输出：
![回收子进程](https://images2018.cnblogs.com/blog/1127869/201807/1127869-20180725095801778-1457292536.png)

---

扩展：（回收所有子进程，status返回-1代表没有子进程了，Python里面没有子进程会触发异常）

```py
import os
import time

def main():
    i = 0
    while i < 3:
        pid = os.fork()
        # 防止产生孙子进程（可以自己思索下）
        if pid == 0:
            break
        i += 1

    if i == 0:
        print("i=%d,子进程：Pid=%d,PPID=%d" % (i, os.getpid(), os.getppid()))
        time.sleep(1)
    elif i == 1:
        print("i=%d,子进程：Pid=%d,PPID=%d" % (i, os.getpid(), os.getppid()))
        time.sleep(1)
    elif i == 2:
        print("i=%d,子进程：Pid=%d,PPID=%d" % (i, os.getpid(), os.getppid()))
        time.sleep(3)
        while 1:
            print("(PID=%d)我又老卵了，怎么滴～" % os.getpid())
            time.sleep(3)
    elif i==3: # 循环结束后，父进程才会退出，这时候i=3
        print("i=%d,父进程：Pid=%d,PPID=%d" % (i, os.getpid(), os.getppid()))
        while True:
            print("等待回收子进程")
            try:
                wpid, status = os.wait()
                print(wpid)
                print(status)
                if status == 0:
                    print("正常退出")
                elif status == 9:
                    print("被信号9干死了")
            except OSError as ex:
                print(ex)
                break

    print("pid=%d,over，ppid=%d" % (os.getpid(), os.getppid()))

if __name__ == '__main__':
    main()
```
演示：看最后一句输出，父进程扫尾工作做完就over了
![回收所有子进程](https://images2018.cnblogs.com/blog/1127869/201807/1127869-20180725105051778-1590933700.png)

---

#### 2.3.2.waitpid

上面说的`wait`方法是阻塞进程的一种方式，`waitpid`可以设置不阻塞进程

```py
help(os.waitpid)

Help on built-in function waitpid in module posix:

waitpid(pid, options, /)
    Wait for completion of a given child process.
    
    Returns a tuple of information regarding the child process:
        (pid, status)
    
    The options argument is ignored on Windows.
```
等待进程id为pid的进程结束，返回一个tuple，包括进程的进程ID和退出信息(和os.wait()一样)，参数options会影响该函数的行为。在默认情况下，options的值为0。

1. 如果pid是一个正数，waitpid()请求获取一个pid指定的进程的退出信息
2. **如果pid为0，则等待并获取当前进程组中的任何子进程的值**
3. 如果pid为-1，则等待当前进程的任何子进程
4. 如果pid小于-1，则获取进程组id为pid的绝对值的任何一个进程
5. 当系统调用返回-1时，抛出一个OSError异常。

options:
```py
os.WNOHANG - 如果没有子进程退出，则不阻塞waitpid()调用
os.WCONTINUED - 如果子进程从stop状态变为继续执行，则返回进程自前一次报告以来的信息。
os.WUNTRACED - 如果子进程被停止过而且其状态信息还没有报告过，则报告子进程的信息。
```

用法和`wait`差不多，就是多了一个不阻塞线程的方法：
```py
import os
import time

def main():
    pid = os.fork()

    if pid == 0:
        print("[子进程]PID：%d，PPID：%d" % (os.getpid(), os.getppid()))
        time.sleep(2)

    elif pid > 0:
        print("[父进程]PID：%d，PPID：%d" % (os.getpid(), os.getppid()))
        while True:
            try:
                wpid, status = os.waitpid(0, os.WNOHANG)
                if wpid > 0:
                    print("回收子进程wpid:%d,状态status:%d" % (wpid, status))
            except OSError:
                print("没有子进程了")
                break

            print("父进程忙着挣钱养家呢～")
            time.sleep(3)

    print("[over]PID：%d，PPID：%d" % (os.getpid(), os.getppid()))

if __name__ == '__main__':
    main()
```
输出：
```
[父进程]PID：1371，PPID：29604
[子进程]PID：1372，PPID：1371
父进程忙着挣钱养家呢～
[over]PID：1372，PPID：1371
回收子进程wpid:1372,状态status:0
父进程忙着挣钱养家呢～
没有子进程了
[over]PID：1371，PPID：29604
```

#### 2.3.3.wait3 and wait4

```py
help(os.wait3)

Help on built-in function wait3 in module posix:

wait3(options)
    Wait for completion of a child process.
    
    Returns a tuple of information about the child process:
      (pid, status, rusage)
```
```py
help(os.wait4)

Help on built-in function wait4 in module posix:

wait4(pid, options)
    Wait for completion of a specific child process.
    
    Returns a tuple of information about the child process:
      (pid, status, rusage)
```
这个是Python扩展的方法，用法和`wait、waitpid`差不多，我就不一个个的举例子了，以`wait3`为例
```py
import os
import time

def main():
    pid = os.fork()

    if pid == 0:
        print("[子进程]PID：%d，PPID：%d" % (os.getpid(), os.getppid()))
        time.sleep(2)

    elif pid > 0:
        print("[父进程]PID：%d，PPID：%d" % (os.getpid(), os.getppid()))
        while True:
            try:
                wpid, status, rusage = os.wait3(os.WNOHANG)
                if wpid > 0:
                    print("回收子进程wpid:%d,状态status:%d\n详细信息：%s" % (wpid, status,
                                                                 rusage))
            except OSError:
                print("没有子进程了")
                break

            print("父进程忙着挣钱养家呢～")
            time.sleep(3)

    print("[over]PID：%d，PPID：%d" % (os.getpid(), os.getppid()))

if __name__ == '__main__':
    main()
```
输出
```
[父进程]PID：2638，PPID：29604
[子进程]PID：2639，PPID：2638
父进程忙着挣钱养家呢～
[over]PID：2639，PPID：2638
回收子进程wpid:2639,状态status:0
详细信息：resource.struct_rusage(ru_utime=0.0052179999999999995, ru_stime=0.0052179999999999995, ru_maxrss=7032, ru_ixrss=0, ru_idrss=0, ru_isrss=0, ru_minflt=869, ru_majflt=0, ru_nswap=0, ru_inblock=0, ru_oublock=0, ru_msgsnd=0, ru_msgrcv=0, ru_nsignals=0, ru_nvcsw=2, ru_nivcsw=0)
父进程忙着挣钱养家呢～
没有子进程了
[over]PID：2638，PPID：29604
```

---

### 扩展：execl and execlp

之前有说fork后，相当于copy了一份，.text里面放的是代码段，如果想要调用另一个程序，可以使用`execlxxx`，他会把.text里面的代码替换掉

```py
help(os.execl)

Help on function execl in module os:

execl(file, *args)
    execl(file, *args)
    
    Execute the executable file with argument list args, replacing the
    current process.
```
```py
help(os.execlp)

Help on function execlp in module os:

execlp(file, *args)
    execlp(file, *args)
    
    Execute the executable file (which is searched for along PATH)
    with argument list args, replacing the current process.
```
来看个例子，`os.execl("绝对路径","参数或者指令")` or `os.execlp("Path中包含的命令","参数或者指令")`

提示：查看命令路径：eg：`which pstree`

```py
import os

def main():
    pid = os.fork()
    if pid == 0:
        # 第二个参数不能为None,，第一个路径为绝对路径 eg：os.execl("/bin/ls"," ")
        os.execl("/usr/bin/pstree", "-ps") # os.execlp("pstree", "-ps")  # 执行Path环境变量可以搜索到的命令

        print("exec函数族会替换代码，我是不会被执行的，除非上面的出问题了")

    print("-" * 10) # 父进程执行一次，子进程不会执行

if __name__ == '__main__':
    main()
```
注意输出信息：
```
----------
systemd(1)─┬─NetworkManager(646)─┬─{NetworkManager}(682)
           │                     └─{NetworkManager}(684)
           ├─agetty(1470)
           ├─auditd(600)───{auditd}(601)
           ├─crond(637)
           ├─dbus-daemon(629)───{dbus-daemon}(634)
           ├─firewalld(645)───{firewalld}(774)
           ├─lvmetad(483)
           ├─master(1059)─┬─pickup(62643)
           │              └─qmgr(1061)
           ├─polkitd(627)─┬─{polkitd}(636)
           │              ├─{polkitd}(639)
           │              ├─{polkitd}(640)
           │              ├─{polkitd}(642)
           │              └─{polkitd}(643)
           ├─rsyslogd(953)─┬─{rsyslogd}(960)
           │               └─{rsyslogd}(961)
           ├─sshd(950)───sshd(66635)───sshd(66651)───bash(66652)───pstree(66678+
           ├─systemd-journal(462)
           ├─systemd-logind(626)
           ├─systemd-udevd(492)
           └─tuned(951)─┬─{tuned}(1005)
                        ├─{tuned}(1006)
                        ├─{tuned}(1007)
                        └─{tuned}(1048)
```

### 2.4.进程间通信～管道

讲管道之前，先说个简单的：通过文件进行通信



#### 2.4.1.进程间通信～PIPE

#### 2.4.2.进程间通信～FIFO

#### 2.4.3.进程间通信～MMAP


## 3.线程相关

In [1]:
!ls

with open("test.txt", "w") as f:
    f.write("从前有座山,山上有座庙,庙里有个老和尚和一个小和尚。有一天,老和尚对小和尚说:")

with open("test.txt", "r") as f:
    data = f.read()
    print(data)

!ls

并发编程～进程先导篇.ipynb
从前有座山,山上有座庙,庙里有个老和尚和一个小和尚。有一天,老和尚对小和尚说:
test.txt  并发编程～进程先导篇.ipynb


Help on function execlp in module os:

execlp(file, *args)
    execlp(file, *args)
    
    Execute the executable file (which is searched for along $PATH)
    with argument list args, replacing the current process.

